# 6. The ColumnTransformer

The `ColumnTransformer` is a completely new meta-estimator released for scikit-learn version 0.20. It has greatly improved the transitioning from pandas to scikit-learn. The addition of the `ColumnTransformer` allows us to:

* Apply different transformations to different columns of data
* Use a pandas dataframe as input data
* Create a pipeline connecting all of our steps

## The previous workflow
Before the `ColumnTransformer` existed, there was no direct way to apply different transformations to different columns of data. This was a major missing ability as string columns are processed differently than numeric columns. One of the most common frustrating scenarios was to have a pandas DataFrame consisting of both string and numeric columns with the desire to do machine learning on it in scikit-learn.

### Common workarounds
Because there was no available path in scikit-learn to prepare a pandas dataframe with a mix of string and numeric columns for machine learning, a number of workarounds were built.

* pandas `get_dummies` function - is able to one-hot encode string columns (and ignore the numeric columns) but it is not an estimator, does not integrate with a pipeline, and cannot be used on unseen data to produce the same encoding.
* scikit-learn's `MultiLabelBinarizer` - This actually does one-hot encoding for strings but was originally built for target variables and not input data. Importantly, you cannot apply it just a subset of the data.
* The pandas [sklearn_pandas][1] library - The integration between pandas and scikit-learn was so bad, that an entire new package was built. It does provide an alternative solution to the integration problems, but ColumnTransformer solves these issues now.

The `ColumnTransformer` paired with the upgraded `OneHotEncoder` makes the transition from pandas much easier, more robust, and gives us a single obvious path forward.

[1]: https://github.com/scikit-learn-contrib/sklearn-pandas

## Processing string and numeric columns separately
String columns are processed differently than numeric columns and need separate pipelines to handle the transformations for each. For instance, we might want to impute missing values with a constant for string columns and the mean for numeric.

In [28]:
import pandas as pd
hs = pd.read_csv('data/housing_sample.csv')
hs.head()

,Neighborhood,Exterior1st,YearBuilt,LotFrontage,GrLivArea,GarageArea,SalePrice
0,CollgCr,VinylSd,2003,65.0,1710,548,208500
1,Other,Other,1976,80.0,1262,460,181500
2,CollgCr,VinylSd,2001,68.0,1786,608,223500
3,Other,Other,1915,60.0,1717,642,140000
4,Other,VinylSd,2000,84.0,2198,836,250000


In [38]:
hs.isna().sum()

Neighborhood     14
Exterior1st      43
YearBuilt         0
LotFrontage     259
GrLivArea         0
GarageArea        0
SalePrice         0
dtype: int64

In [50]:
y = hs.pop('SalePrice').values

## Using the `ColumnTransformer`
The major purpose of the `ColumnTransformer` is to apply a transformation to a specific subset of the columns and not the entire input data which is done for all the other transformers. Let's say we want to only fill in missing values for the string columns. 

To get started, we must create a list of three-item tuples. The tuple items are the name of the transformer (as a string), the instantiated transformer, and the list of columns to apply the transformation to. The `ColumnTransformer` is built to be used with pandas DataFrames so you can pass in your DataFrame directly to it.

Below, we create a list with a single three-item tuple to impute missing values to the 'Neighborhood' and 'Exterior1st' string columns

In [37]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
si = SimpleImputer(strategy='constant', fill_value='MISSING')
string_cols = ['Neighborhood', 'Exterior1st']
transformers = [('impute', si, string_cols)]

We can now instantiate the `ColumnTransformer` with our list of transformations and then fit and return the transformed data.

In [39]:
ct = ColumnTransformer(transformers)
X_filled = ct.fit_transform(hs)
X_filled

array([['CollgCr', 'VinylSd'],
       ['Other', 'Other'],
       ['CollgCr', 'VinylSd'],
       ...,
       ['Other', 'Other'],
       ['NAmes', 'Other'],
       ['Edwards', 'HdBoard']], dtype=object)

### Columns dropped - numpy array returned
By default, all the columns not listed are dropped. Also, we are returned a numpy array and are no longer in pandas.

### Keep the remaining columns
The `remainder` parameter controls what happens to the other columns not specified in the list of transformers. This parameter is defaulted to the string 'drop'. Setting it to 'passthrough' will keep the other columns in the returned array. 

In [51]:
ct = ColumnTransformer(transformers, remainder='passthrough')
ct.fit_transform(hs)[:5]

array([['CollgCr', 'VinylSd', 2003.0, 65.0, 1710.0, 548.0],
       ['Other', 'Other', 1976.0, 80.0, 1262.0, 460.0],
       ['CollgCr', 'VinylSd', 2001.0, 68.0, 1786.0, 608.0],
       ['Other', 'Other', 1915.0, 60.0, 1717.0, 642.0],
       ['Other', 'VinylSd', 2000.0, 84.0, 2198.0, 836.0]], dtype=object)

### Imputing the numeric columns with the mean
The `ColumnTransformer` allows us to use different imputation strategies for different sets of columns. Below, we instantiate a second imputer and use it for the numeric columns. It's not necessary to use 'passthrough' as all the columns are named in the transformers.

In [54]:
string_si = SimpleImputer(strategy='constant', fill_value='MISSING')
numeric_si = SimpleImputer(strategy='mean')

string_cols = ['Neighborhood', 'Exterior1st']
numeric_cols = ['YearBuilt', 'LotFrontage', 'GrLivArea', 'GarageArea']

transformers = [('impute_string', string_si, string_cols), 
                ('impute_numeic', numeric_si, numeric_cols)]

ct = ColumnTransformer(transformers)
ct.fit_transform(hs)[:5]

array([['CollgCr', 'VinylSd', 2003.0, 65.0, 1710.0, 548.0],
       ['Other', 'Other', 1976.0, 80.0, 1262.0, 460.0],
       ['CollgCr', 'VinylSd', 2001.0, 68.0, 1786.0, 608.0],
       ['Other', 'Other', 1915.0, 60.0, 1717.0, 642.0],
       ['Other', 'VinylSd', 2000.0, 84.0, 2198.0, 836.0]], dtype=object)

## Build separate pipelines
If we want to apply multiple transformations to different subsets of the data then we will need to a build a pipeline for each section. Below, we impute missing values and one-hot encode the string columns and separately impute the missing values and standardize the numeric columns.

In [69]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# string pipeline
string_si = SimpleImputer(strategy='constant', fill_value='MISSING')
ohe = OneHotEncoder(sparse=False)
steps = [('impute', string_si), ('encode', ohe)]
string_pipe = Pipeline(steps)

# numeric pipeline
numeric_si = SimpleImputer(strategy='mean')
ss = StandardScaler()
steps = [('si', numeric_si), ('standardize', ss)]
numeric_pipe = Pipeline(steps)

# columns
string_cols = ['Neighborhood', 'Exterior1st']
numeric_cols = ['YearBuilt', 'LotFrontage', 'GrLivArea', 'GarageArea']

transformers = [('string', string_pipe, string_cols), 
                ('numeric', numeric_pipe, numeric_cols)]

ct = ColumnTransformer(transformers)
X_transformed = ct.fit_transform(hs)
X_transformed

array([[ 1.        ,  0.        ,  0.        , ..., -0.22937175,
         0.37033344,  0.35100032],
       [ 0.        ,  0.        ,  0.        , ...,  0.4519361 ,
        -0.48251191, -0.06073101],
       [ 1.        ,  0.        ,  0.        , ..., -0.09311018,
         0.51501256,  0.63172623],
       ...,
       [ 0.        ,  0.        ,  0.        , ..., -0.18395123,
         1.56964722, -1.03391416],
       [ 0.        ,  0.        ,  0.        , ..., -0.09311018,
        -0.83278768, -1.09005935],
       [ 0.        ,  1.        ,  0.        , ...,  0.22483348,
        -0.49393395, -0.9216238 ]])

## Adding machine learning
We can create one last pipeline to do machine learning on the final transformed output.

In [108]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, KFold
from mymetrics import root_mean_squared_log_error

rfr = RandomForestRegressor()
steps = [('transformers', ct), ('rfr', rfr)]
final_pipe = Pipeline(steps)

kf = KFold(n_splits=5, shuffle=True)
cross_val_score(final_pipe, hs, y, cv=kf, scoring=root_mean_squared_log_error)

array([-0.18375499, -0.20026133, -0.22439418, -0.17708722, -0.20331659])

## Grid searching a pipeline of transformers
To access a specific estimator in this pipeline of transformers, you must continually append the name of the transformer/pipeline followed by two underscores.

In [107]:
from sklearn.model_selection import GridSearchCV
grid = {'transformers__numeric__si__strategy': ['mean', 'median'],
       'rfr__n_estimators': [50, 100, 500], 'rfr__max_depth': range(2, 10)}
gs = GridSearchCV(final_pipe, grid, cv=kf, scoring=root_mean_squared_log_error)
gs.fit(hs, y)
gs.best_params_

{'rfr__max_depth': 9,
 'rfr__n_estimators': 500,
 'transformers__numeric__si__strategy': 'median'}

## Summary of Commands

In [110]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV
from mymetrics import root_mean_squared_log_error

# string pipeline
string_si = SimpleImputer(strategy='constant', fill_value='MISSING')
ohe = OneHotEncoder(sparse=False)
steps = [('impute', string_si), ('encode', ohe)]
string_pipe = Pipeline(steps)

# numeric pipeline
numeric_si = SimpleImputer(strategy='mean')
ss = StandardScaler()
steps = [('si', numeric_si), ('standardize', ss)]
numeric_pipe = Pipeline(steps)

# columns
string_cols = ['Neighborhood', 'Exterior1st']
numeric_cols = ['YearBuilt', 'LotFrontage', 'GrLivArea', 'GarageArea']

transformers = [('string', string_pipe, string_cols), 
                ('numeric', numeric_pipe, numeric_cols)]

ct = ColumnTransformer(transformers)
rfr = RandomForestRegressor()
steps = [('transformers', ct), ('rfr', rfr)]
final_pipe = Pipeline(steps)

kf = KFold(n_splits=5, shuffle=True)
grid = {'transformers__numeric__si__strategy': ['mean', 'median'],
       'rfr__n_estimators': [50, 100], 'rfr__max_depth': range(2, 6)}
gs = GridSearchCV(final_pipe, grid, cv=kf, scoring=root_mean_squared_log_error)
gs.fit(hs, y)
gs.best_params_

{'rfr__max_depth': 5,
 'rfr__n_estimators': 50,
 'transformers__numeric__si__strategy': 'median'}

## Exercise
Use the `ColumnTransformer` to build separate pipelines for string and numeric columns. Build a final pipeline that adds machine learning as the last step.